 <h1>                              Melody Generator using Second Order Markov Chain </h1>

  <h6> What are Midi Files? </h6> <br>
  MIDI stands for Musical Instrument Digital Interface. MIDI files are a format for creating, sharing, and playing digital music. MIDI files are binary files that contain instructions for pitch and tempo, but not actual audio data. 
<br> Midi files store each song as different tracks. There are notes in each track that make the music, it has the following parameters : <br> 
<ol> 
    <li><b>pitch</b> - how high the frequency is , 1-120 values are allowed in Midi Files</li>
    <li><b>Velocity</b> - how lound the note is going to sound</li>
    <li><b>Time On/Off</b> - at what time the note will be played and stopped</li>
</ol>


<h6> What are Second Order Markov chains? </h6> <br>
 Second-order Markov chains allow us to model situations where what happens next depends on both the <b>current state</b> and the <b>one just before</b> it. It's like making decisions based on a bit more history, considering the recent past along with the present to predict the future.

In [1]:
import mido
import numpy as np
import random
import pretty_midi
import pygame
from mido import MidiFile, MidiTrack, Message
import copy
import math

pygame 2.5.2 (SDL 2.28.3, Python 3.9.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
NUMBER_OF_FILES = 100
SIZE_OF_MUSIC = 15000

In [3]:
# open the files by reading the metadata file
def get_names(filepath):
    with open(filepath) as metadata:
        lines = metadata.readlines()
    
    names = []
    for i in range(len(lines)):
        midi_name = lines[i].split("\t")[0]
        names.append(f"midi/{midi_name}.mid")
    return names


<h4>The following function is used to traverse the Midi file and create an array called events that stores the info about each note </h4>

In [4]:
def parse_midi(file_path):
#     print(file_path)
    midi_data = mido.MidiFile(file_path)
    events = []
    start_time =0
    on_time = 0
    for track in midi_data.tracks:
        for msg in track:
            if msg.type == 'note_on' or msg.type == 'note_off':
                if msg.type == 'note_on':
                    start = msg.time
                    on_time = start
                if msg.type == 'note_off':
                    on_time = msg.time - start_time
                # Extract relevant information, e.g., note, velocity, time
#                 print("note : " , msg.note)
                events.append((msg.note, msg.velocity, on_time))

    return events

For a second degree Markov chain we need to create a 3D matrix.<br>
to create the matrix we first separate all the unique states from the data , and map then using a list called "state_mapping".<br> basically the unique states array contains a 3-tuple and the state_mapping array contains their index<br>
Then we how many times each transition happened and store it in the matrix

In [5]:
def build_transition_matrix(data): # only one parameter , no need for num_states as they are going to be calculated later , data is cumulative data
    # get all the unique states and give them 1 integer value to later create a transition matrix
    unique_states , state_mapping = np.unique(data ,axis = 0, return_inverse = True)
    
    num_states = len(unique_states)

    transition_counts = np.full((num_states , num_states , num_states) , 0.00001)
#     print("len of states : " , len(states_sequence))

    for i in range(1 ,len(data)):
        current_state = state_mapping[i]
        previous_state = state_mapping[i-1]
        second_previous_state = state_mapping[i-2]
        transition_counts[current_state,previous_state,second_previous_state]+=1

    return transition_counts, unique_states

Normalising the transition matrix to create probabilities

In [6]:
def normalisation(cumulative_transition_matrix):
    row_sums = cumulative_transition_matrix.sum(axis=1, keepdims=True)

    normalised_transition_matrix = np.divide(cumulative_transition_matrix,row_sums)
    normalised_transition_matrix = np.nan_to_num(normalised_transition_matrix, nan=0.00001)
    
    return normalised_transition_matrix

In [7]:
#find the index of note with the highest probability:
def find_max_index(transition_matrix):
    flat_index_max = np.nanargmax(transition_matrix)
    max_val = np.nanmax(transition_matrix)
    print("max value is " , max_val)
    row_index, col_index, z_index = np.unravel_index(flat_index_max, transition_matrix.shape)
    return row_index

Now ato generate music we can traverse the transition_matrix (the probabilities that are stored in the matrix)

In [8]:
def generate_music(transition_matrix, length):
    num_states = transition_matrix.shape[0]
    
    #starting with the note that occurs the most in the given data set
    current_state = find_max_index(transition_matrix)
    print("start state =", current_state)
    music_sequence = [current_state]
    music_sequence.append(current_state)

    for i in range(2,length - 1):
        try:
            print("current state:", current_state)
            probabilities = transition_matrix[music_sequence[i-1],:, music_sequence[i-2]]
            print("size of p= " , probabilities.shape)
            next_state = np.random.choice(np.arange(num_states), p=probabilities)
            current_state = math.floor(math.sqrt(next_state))
            music_sequence.append(current_state)
            
        except:
            probabilities = transition_matrix[music_sequence[i-1],:,music_sequence[i-2]]
            next_state = np.random.choice(np.arange(num_states), p=probabilities)
            current_state = math.floor(math.sqrt(next_state))
            music_sequence.append(current_state)
            
    return music_sequence

Now i open the files and create my normalised transition matrix

In [9]:
#openening files and creating cumulative transition matrix

names = get_names("MTC-LC-labels.txt")
data = []
file_number = 0
for file_name in names:
    file_number+=1
    if file_number == NUMBER_OF_FILES:
        break
    for event in parse_midi(file_name):
        data.append(event)
print("total files read = " , file_number)

cumulative_transition_matrix , unique_states= build_transition_matrix(data)
normalised_transition_matrix = normalisation(cumulative_transition_matrix)

total files read =  100


Then i generate the new Midi file using the normalised transition matrix

In [14]:
music_sequence = generate_music(normalised_transition_matrix , SIZE_OF_MUSIC)
print(len(music_sequence))

mid = MidiFile()
track = MidiTrack()
mid.tracks.append(track)
current_time = np.random.choice(128)

print("random state = " ,music_sequence[4])
print("note at that state = " , unique_states[4])
on_time = 0
for state in music_sequence:
    note_values = unique_states[state]
    on_time = note_values[2]
    note_start = current_time+1
    note_end = current_time+on_time
    note_on = Message('note_on', note=note_values[0], velocity=note_values[1], time=note_start)
    note_off = Message('note_off', note=note_values[0], velocity=note_values[1], time=note_end)
    track.append(note_on)
    track.append(note_off)
    
mid.save('new_output.mid')

max value is  0.9992751417432428
start state = 382
current state: 382
size of p=  (944,)
current state: 20
size of p=  (944,)
current state: 14
size of p=  (944,)
current state: 13
size of p=  (944,)
current state: 24
size of p=  (944,)
current state: 28
size of p=  (944,)
current state: 28
size of p=  (944,)
current state: 8
size of p=  (944,)
current state: 17
size of p=  (944,)
current state: 16
size of p=  (944,)
current state: 14
size of p=  (944,)
current state: 12
size of p=  (944,)
current state: 18
size of p=  (944,)
current state: 30
size of p=  (944,)
current state: 12
size of p=  (944,)
current state: 23
size of p=  (944,)
current state: 20
size of p=  (944,)
current state: 11
size of p=  (944,)
current state: 23
size of p=  (944,)
current state: 27
size of p=  (944,)
current state: 28
size of p=  (944,)
current state: 23
size of p=  (944,)
current state: 29
size of p=  (944,)
current state: 10
size of p=  (944,)
current state: 22
size of p=  (944,)
current state: 19
size o

current state: 18
size of p=  (944,)
current state: 17
size of p=  (944,)
current state: 20
size of p=  (944,)
current state: 30
size of p=  (944,)
current state: 27
size of p=  (944,)
current state: 30
size of p=  (944,)
current state: 22
size of p=  (944,)
current state: 9
size of p=  (944,)
current state: 27
size of p=  (944,)
current state: 21
size of p=  (944,)
current state: 11
size of p=  (944,)
current state: 27
size of p=  (944,)
current state: 29
size of p=  (944,)
current state: 26
size of p=  (944,)
current state: 28
size of p=  (944,)
current state: 25
size of p=  (944,)
current state: 15
size of p=  (944,)
current state: 24
size of p=  (944,)
current state: 12
size of p=  (944,)
current state: 20
size of p=  (944,)
current state: 18
size of p=  (944,)
current state: 22
size of p=  (944,)
current state: 26
size of p=  (944,)
current state: 17
size of p=  (944,)
current state: 23
size of p=  (944,)
current state: 12
size of p=  (944,)
current state: 29
size of p=  (944,)
cu

current state: 21
size of p=  (944,)
current state: 23
size of p=  (944,)
current state: 2
size of p=  (944,)
current state: 22
size of p=  (944,)
current state: 24
size of p=  (944,)
current state: 9
size of p=  (944,)
current state: 17
size of p=  (944,)
current state: 14
size of p=  (944,)
current state: 20
size of p=  (944,)
current state: 26
size of p=  (944,)
current state: 30
size of p=  (944,)
current state: 23
size of p=  (944,)
current state: 4
size of p=  (944,)
current state: 27
size of p=  (944,)
current state: 10
size of p=  (944,)
current state: 17
size of p=  (944,)
current state: 16
size of p=  (944,)
current state: 25
size of p=  (944,)
current state: 13
size of p=  (944,)
current state: 23
size of p=  (944,)
current state: 15
size of p=  (944,)
current state: 12
size of p=  (944,)
current state: 27
size of p=  (944,)
current state: 28
size of p=  (944,)
current state: 28
size of p=  (944,)
current state: 21
size of p=  (944,)
current state: 7
size of p=  (944,)
curre

current state: 15
size of p=  (944,)
current state: 19
size of p=  (944,)
current state: 9
size of p=  (944,)
current state: 27
size of p=  (944,)
current state: 25
size of p=  (944,)
current state: 4
size of p=  (944,)
current state: 3
size of p=  (944,)
current state: 28
size of p=  (944,)
current state: 22
size of p=  (944,)
current state: 28
size of p=  (944,)
current state: 11
size of p=  (944,)
current state: 29
size of p=  (944,)
current state: 16
size of p=  (944,)
current state: 24
size of p=  (944,)
current state: 19
size of p=  (944,)
current state: 29
size of p=  (944,)
current state: 26
size of p=  (944,)
current state: 30
size of p=  (944,)
current state: 16
size of p=  (944,)
current state: 15
size of p=  (944,)
current state: 29
size of p=  (944,)
current state: 9
size of p=  (944,)
current state: 26
size of p=  (944,)
current state: 24
size of p=  (944,)
current state: 14
size of p=  (944,)
current state: 29
size of p=  (944,)
current state: 25
size of p=  (944,)
curre

current state: 14
size of p=  (944,)
current state: 19
size of p=  (944,)
current state: 8
size of p=  (944,)
current state: 15
size of p=  (944,)
current state: 15
size of p=  (944,)
current state: 29
size of p=  (944,)
current state: 12
size of p=  (944,)
current state: 14
size of p=  (944,)
current state: 18
size of p=  (944,)
current state: 13
size of p=  (944,)
current state: 25
size of p=  (944,)
current state: 21
size of p=  (944,)
current state: 8
size of p=  (944,)
current state: 30
size of p=  (944,)
current state: 19
size of p=  (944,)
current state: 21
size of p=  (944,)
current state: 15
size of p=  (944,)
current state: 26
size of p=  (944,)
current state: 15
size of p=  (944,)
current state: 21
size of p=  (944,)
current state: 18
size of p=  (944,)
current state: 23
size of p=  (944,)
current state: 25
size of p=  (944,)
current state: 16
size of p=  (944,)
current state: 2
size of p=  (944,)
current state: 29
size of p=  (944,)
current state: 22
size of p=  (944,)
curr

...

current state: 7
size of p=  (944,)
current state: 8
size of p=  (944,)
current state: 17
size of p=  (944,)
current state: 11
size of p=  (944,)
current state: 26
size of p=  (944,)
current state: 11
size of p=  (944,)
current state: 24
size of p=  (944,)
current state: 24
size of p=  (944,)
current state: 25
size of p=  (944,)
current state: 10
size of p=  (944,)
current state: 11
size of p=  (944,)
current state: 12
size of p=  (944,)
current state: 18
size of p=  (944,)
current state: 15
size of p=  (944,)
current state: 26
size of p=  (944,)
current state: 10
size of p=  (944,)
current state: 14
size of p=  (944,)
current state: 30
size of p=  (944,)
current state: 14
size of p=  (944,)
current state: 25
size of p=  (944,)
current state: 28
size of p=  (944,)
current state: 30
size of p=  (944,)
current state: 27
size of p=  (944,)
current state: 27
size of p=  (944,)
current state: 24
size of p=  (944,)
current state: 27
size of p=  (944,)
current state: 21
size of p=  (944,)
cur

current state: 14
size of p=  (944,)
current state: 22
size of p=  (944,)
current state: 22
size of p=  (944,)
current state: 11
size of p=  (944,)
current state: 25
size of p=  (944,)
current state: 25
size of p=  (944,)
current state: 7
size of p=  (944,)
current state: 19
size of p=  (944,)
current state: 30
size of p=  (944,)
current state: 27
size of p=  (944,)
current state: 28
size of p=  (944,)
current state: 30
size of p=  (944,)
current state: 30
size of p=  (944,)
current state: 13
size of p=  (944,)
current state: 30
size of p=  (944,)
current state: 9
size of p=  (944,)
current state: 24
size of p=  (944,)
current state: 8
size of p=  (944,)
current state: 16
size of p=  (944,)
current state: 21
size of p=  (944,)
current state: 13
size of p=  (944,)
current state: 30
size of p=  (944,)
current state: 20
size of p=  (944,)
current state: 23
size of p=  (944,)
current state: 18
size of p=  (944,)
current state: 15
size of p=  (944,)
current state: 28
size of p=  (944,)
curr

current state: 14
size of p=  (944,)
current state: 3
size of p=  (944,)
current state: 8
size of p=  (944,)
current state: 29
size of p=  (944,)
current state: 13
size of p=  (944,)
current state: 29
size of p=  (944,)
current state: 27
size of p=  (944,)
current state: 24
size of p=  (944,)
current state: 4
size of p=  (944,)
current state: 17
size of p=  (944,)
current state: 18
size of p=  (944,)
current state: 24
size of p=  (944,)
current state: 27
size of p=  (944,)
current state: 13
size of p=  (944,)
current state: 29
size of p=  (944,)
current state: 8
size of p=  (944,)
current state: 16
size of p=  (944,)
current state: 23
size of p=  (944,)
current state: 6
size of p=  (944,)
current state: 21
size of p=  (944,)
current state: 28
size of p=  (944,)
current state: 17
size of p=  (944,)
current state: 25
size of p=  (944,)
current state: 22
size of p=  (944,)
current state: 19
size of p=  (944,)
current state: 27
size of p=  (944,)
current state: 5
size of p=  (944,)
current

current state: 13
size of p=  (944,)
current state: 28
size of p=  (944,)
current state: 19
size of p=  (944,)
current state: 9
size of p=  (944,)
current state: 12
size of p=  (944,)
current state: 16
size of p=  (944,)
current state: 9
size of p=  (944,)
current state: 2
size of p=  (944,)
current state: 25
size of p=  (944,)
current state: 16
size of p=  (944,)
current state: 26
size of p=  (944,)
current state: 14
size of p=  (944,)
current state: 24
size of p=  (944,)
current state: 22
size of p=  (944,)
current state: 8
size of p=  (944,)
current state: 26
size of p=  (944,)
current state: 9
size of p=  (944,)
current state: 3
size of p=  (944,)
current state: 27
size of p=  (944,)
current state: 15
size of p=  (944,)
current state: 24
size of p=  (944,)
current state: 14
size of p=  (944,)
current state: 20
size of p=  (944,)
current state: 30
size of p=  (944,)
current state: 29
size of p=  (944,)
current state: 14
size of p=  (944,)
current state: 27
size of p=  (944,)
current

random state =  13
note at that state =  [ 57  65 480]


using the following code we can listen to the new Midi file created!!!

In [16]:
#playing a midi file 

pygame.init()

midi_file = 'new_output.mid'
pygame.mixer.music.load(midi_file)
pygame.mixer.music.play()

pygame.time.wait(15000)

pygame.quit()


<br>
<br>
<br>
<h2> Research papers and data set used </h2>
<br>
<a href = "https://hajim.rochester.edu/ece/sites/zduan/teaching/ece477/projects/2019/MichaelFinley_GavinBaker_MelodyGeneration_FinalReport.pdf"><b>Research paper 1 </b></a>
<br>
<br>
<a href = "https://www.cs.cmu.edu/~shkarp/course_projects/music_generation.pdf">
<b>Research Paper 2</b> </a> 
<br>
<br> <a href = "https://www.liederenbank.nl/mtc/#">
<b> dataset used (MTC-LC-1.0) </b></a>